In [1]:
import numpy as np
import tensorflow as tf


In [2]:
####
# Sequence Labeling with variable-length sequences
####

import os
from datetime import datetime

tf.reset_default_graph()

NUM_HIDDEN = 5
NUM_CLASS = 3
FEATURE_SIZE_PER_TIMESTEP = 5

### Data pipeline
def input_pipeline(filename, batch_size, epochs=None):
    file_list = [os.path.join(os.getcwd(), filename)]
    file_queue = tf.train.string_input_producer(file_list, num_epochs=epochs)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(file_queue)
    sequence_features = {
        "inputs": tf.FixedLenSequenceFeature([FEATURE_SIZE_PER_TIMESTEP], dtype=tf.float32),
        "label": tf.FixedLenSequenceFeature([], dtype=tf.int64)
    }
    _, sequence = tf.parse_single_sequence_example(
        serialized=serialized_example,
        sequence_features=sequence_features)

    actual_length = tf.shape(sequence["inputs"])[0]
    batch_lengths, batch_sequences, batch_labels = tf.train.batch(
        [actual_length, sequence["inputs"], sequence["label"]],
        batch_size=batch_size,
        dynamic_pad=True,
        allow_smaller_final_batch=True,
        name="input_batching")
    return batch_lengths, batch_sequences, batch_labels


def _last_relevant(outputs, actual_lengths):
    """
    :param outputs: [batch_size x max_seq_length x hidden_size] tensor of dynamic_rnn outputs
    :param actual_lengths: [batch_size] tensor of sequence actual lengths
    :return: [batch_size x hidden_size] tensor of last outputs
    """
    batch_size, max_seq_length, hidden_size = tf.unstack(tf.shape(outputs))
    index = tf.range(0, batch_size) * max_seq_length + (actual_lengths - 1)
    return tf.gather(tf.reshape(outputs, [-1, hidden_size]), index)


### Build Model
def inference(inputs, actual_lengths):
    cell = tf.contrib.rnn.LSTMCell(NUM_HIDDEN)
    outputs, current_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, sequence_length=actual_lengths)
    last_outputs = _last_relevant(outputs, actual_lengths)
    # Output layer weights & biases
    weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN, NUM_CLASS]), dtype=tf.float32)
    biases = tf.Variable(tf.constant(0.1, shape=[NUM_CLASS]), dtype=tf.float32)
    # Softmax classification based on outputs of the last time step of each sequence
    logits = tf.add(tf.matmul(last_outputs, weights), biases)
    predictions = tf.nn.softmax(logits)
    return logits, predictions


## Cost function
def loss(logits, labels, actual_lengths):
    labels_flat = tf.reshape(labels, [-1])
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels_flat)
    mean_loss = tf.reduce_mean(cross_entropy)
    return mean_loss


## Error tracking 
def error(predictions, labels, actual_lengths):
    labels_flat = tf.reshape(labels, [-1])
    errors = tf.not_equal(tf.argmax(predictions, 1))
    mean_error = tf.reduce_mean(tf.cast(errors, tf.float32))
    return mean_error
    

def training(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)    
    return train_op


### Training
NUM_EPOCHS = 100
BATCH_SIZE = 3
DISPLAY_STEP = 5
LEARNING_RATE = 1e-3
TRAINING_SET_SIZE = 7

filename = 'Sequence_classification.tfr'
with tf.Graph().as_default():
    tf.set_random_seed(10)
    np.random.seed(10)    
    # Build Graph
    lengths, sequences, labels = input_pipeline(filename, BATCH_SIZE)
    logits, _ = inference(sequences, lengths)
    avg_loss = loss(logits, labels, lengths)
    train_op = training(avg_loss, LEARNING_RATE)
    
    # Create & Initialize Session
    sess = tf.Session()
    init_op = tf.group(tf.global_variables_initializer(),
                       tf.local_variables_initializer())
    sess.run(init_op)
    # Start QueueRunner
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    try: 
        # Training cycles
        for epoch in range(1, NUM_EPOCHS+1):
            epoch_avg_loss = 0.0
            total_batch = int(TRAINING_SET_SIZE / BATCH_SIZE
                ) + 1 if TRAINING_SET_SIZE % BATCH_SIZE != 0 else int(
                TRAINING_SET_SIZE / BATCH_SIZE)
            for step in range(1, total_batch +1):
                if coord.should_stop():
                    break
                _, train_loss = sess.run([train_op, avg_loss]) 
                epoch_avg_loss += train_loss / total_batch
                assert not np.isnan(train_loss), 'Model diverged with loss = NaN'
                
                if step % DISPLAY_STEP == 0:
                    print('%s: epoch %d, step %d, train_loss = %.6f'
                        % (datetime.now(), epoch, step, train_loss))
                
            print('%s: epoch %d avg_loss = %.6f'
                % (datetime.now(), epoch, epoch_avg_loss))
                
    except tf.errors.OutOfRangeError as e:
        print(e.error_code, e.message)
        print('Done!')
    
    finally:
        coord.request_stop()
    
    coord.join(threads)
    sess.close()
    
print('Finished!')

/Users/Winston/.pyenv/versions/2.7.10/envs/tf-p27/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2017-04-24 14:40:02.554814: epoch 1 avg_loss = 1.408762
2017-04-24 14:40:02.599809: epoch 2 avg_loss = 1.509718
2017-04-24 14:40:02.641374: epoch 3 avg_loss = 1.442610
2017-04-24 14:40:02.673496: epoch 4 avg_loss = 1.424193
2017-04-24 14:40:02.706045: epoch 5 avg_loss = 1.407049
2017-04-24 14:40:02.738983: epoch 6 avg_loss = 1.450705
2017-04-24 14:40:02.779327: epoch 7 avg_loss = 1.373886
2017-04-24 14:40:02.811267: epoch 8 avg_loss = 1.275502
2017-04-24 14:40:02.846800: epoch 9 avg_loss = 1.376903
2017-04-24 14:40:02.894188: epoch 10 avg_loss = 1.300501
2017-04-24 14:40:02.927664: epoch 11 avg_loss = 1.309860
2017-04-24 14:40:02.960635: epoch 12 avg_loss = 1.257352
2017-04-24 14:40:02.996389: epoch 13 avg_loss = 1.319000
2017-04-24 14:40:03.046167: epoch 14 avg_loss = 1.227976
2017-04-24 14:40:03.109553: epoch 15 avg_loss = 1.151765
2017-04-24 14:40:03.160921: epoch 16 avg_loss = 1.243197
2017-04-24 14:40:03.208392: epoch 17 avg_loss = 1.163385
2017-04-24 14:40:03.258136: epoch 18 avg